In [1]:
#安裝套件
!pip install pyserini

In [2]:
import pandas as pd
from google.colab import drive

#檢查資料
drive.mount('/content/drive')
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/IR/train.txt', sep="\n#", header=None)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<ipython-input-2-bf872ee8de7e>:6: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/IR/train.txt', sep="\n#", header=None)


In [3]:
import json
import os.path
from tqdm import tqdm

pubmed_df = pd.DataFrame(columns=['id', 'content'])
count = 0
content = ""
id = ""
for i in tqdm(range(len(data[0]))):
  row = data[0][i]
  # 處理特殊字元
  if '###' in row:
    id = row.split('###')[1]
    content = ""
  # 單字整合成content
  else:
    row = row.replace("\t"," ")
    row = row.split(' ')[1:]

    for word in row:
      content += (word + " ")

    if i == len(data[0]) - 1 or '###' in data[0][i+1]:
      output = {
          "id": id,
          "contents": content
      }

      file_path = 'tests/resources/'+ str(id) +'.json'

      # 確保資料夾存在，如果不存在則創建
      os.makedirs(os.path.dirname(file_path), exist_ok=True)

      # 將資料寫入 JSON 檔案
      with open(file_path, "w") as t:
          json.dump(output, t)
      count += 1



100%|██████████| 2402515/2402515 [02:12<00:00, 18141.54it/s]


1. 使用 Python 命令執行 Pyserini 中的 index.lucene 模組
1. 指定要索引的文檔類型為 JSON 格式
1. 指定要索引的文檔目錄
1. 指定索引的輸出目錄和名稱
1. 指定文檔索引時使用的生成器。在此情況下，使用默認的 Lucene 文檔生成器
1. 指定用於索引的線程數。在此情況下，使用單線程
1. 指定存儲索引中的位置信息、文檔向量和原始文檔的選項


In [4]:
!python -m pyserini.index.lucene \
  --collection JsonCollection \
  --input tests/resources \
  --index indexes/sample_collection_jsonl \
  --generator DefaultLuceneDocumentGenerator \
  --threads 1 \
  --storePositions --storeDocvectors --storeRaw

!pip install faiss-cpu
!apt-get purge openjdk-\* -qq > /dev/null
!apt-get autoremove -qq > /dev/null
!apt-get autoclean -qq > /dev/null
!apt-get install openjdk-21-jdk-headless -qq > /dev/nul
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-21-openjdk-amd64"

2024-04-17 06:41:34,645 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:204) - Setting log level to INFO
2024-04-17 06:41:34,648 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:207) - ============ Loading Index Configuration ============
2024-04-17 06:41:34,648 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:208) - AbstractIndexer settings:
2024-04-17 06:41:34,648 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:209) -  + DocumentCollection path: tests/resources
2024-04-17 06:41:34,649 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:210) -  + CollectionClass: JsonCollection
2024-04-17 06:41:34,649 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:211) -  + Index path: indexes/sample_collection_jsonl
2024-04-17 06:41:34,650 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:212) -  + Threads: 1
2024-04-17 06:41:34,650 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:213) -  + Optimize (merge segments)? false
Apr 17, 2024 6

In [16]:
from pyserini.search.lucene import LuceneSearcher
import time

# 初始化 LuceneSearcher 並指定索引目錄
searcher = LuceneSearcher('indexes/sample_collection_jsonl')

start = time.time()
# 進行搜索，搜索詞為 'cancer'，返回前 100 個結果
hits = searcher.search('AIDS', k=10)
print(hits)
end = time.time()

for i in range(len(hits)):
    # 獲取文檔的檔名，此處假設檔名與文檔 ID 相同，並且位於指定目錄下的 resources 資料夾中
    filename = '/content/tests/resources/'+str(hits[i].docid)+'.json'
    f = open (filename, "r")

    data = json.loads(f.read())
    # 輸出搜索結果的文檔 ID、得分以及對應的內容
    print(f'{i+1:2} {hits[i].docid:4} {hits[i].score:.5f} \n {data}')



[<io.anserini.search.ScoredDoc at 0x7a98a2aff420 jclass=io/anserini/search/ScoredDoc jself=<LocalRef obj=0x5a2a73ccc67a at 0x7a98a2ca9a70>>, <io.anserini.search.ScoredDoc at 0x7a98a2aff1a0 jclass=io/anserini/search/ScoredDoc jself=<LocalRef obj=0x5a2a73ccc682 at 0x7a98a2ca9ad0>>, <io.anserini.search.ScoredDoc at 0x7a98a2aff150 jclass=io/anserini/search/ScoredDoc jself=<LocalRef obj=0x5a2a73ccc68a at 0x7a98a2ca9770>>, <io.anserini.search.ScoredDoc at 0x7a98a2affce0 jclass=io/anserini/search/ScoredDoc jself=<LocalRef obj=0x5a2a73ccc692 at 0x7a98a2ca9d90>>, <io.anserini.search.ScoredDoc at 0x7a98a2aff330 jclass=io/anserini/search/ScoredDoc jself=<LocalRef obj=0x5a2a73ccc69a at 0x7a98a2ca9f70>>, <io.anserini.search.ScoredDoc at 0x7a98a2aff920 jclass=io/anserini/search/ScoredDoc jself=<LocalRef obj=0x5a2a73ccc6a2 at 0x7a98a2caa010>>, <io.anserini.search.ScoredDoc at 0x7a98a2aff880 jclass=io/anserini/search/ScoredDoc jself=<LocalRef obj=0x5a2a73ccc6aa at 0x7a98a2caa190>>, <io.anserini.search

In [11]:
time_InvertedIndex = end - start
print("執行時間：%f 秒" % (time_InvertedIndex))

執行時間：0.066478 秒


In [17]:
import glob  # 導入 glob 模組，用於查找符合指定模式的檔案路徑

searchKey = 'AIDS'

results = []  # 建立空的列表，用於儲存符合條件的資料
start = time.time()

# 使用 glob 模組找到指定目錄下所有以 .json 結尾的檔案路徑
all_file = glob.glob('/content/tests/resources/*.json')

for filename in all_file:
    with open(filename, 'r') as f:
        json_data = json.load(f)
        content = json_data['contents']
        # 檢查搜索關鍵字是否在內容中
        if searchKey in content:
            results.append(json_data)

end = time.time()

for item in results[:10]:
    print(item)
    print()

{'id': '16511419', 'contents': 'To assess the efficacy of an innovative smoking cessation intervention targeted to a multiethnic , economically disadvantaged HIV-positive population . A two-group randomized clinical trial compared a smoking cessation intervention delivered by cellular telephone with usual care approach . Current smokers from a large , inner city HIV/AIDS care center were recruited and randomized to receive either usual care or a cellular telephone intervention . The usual care group received brief physician advice to quit smoking , targeted self-help written materials and nicotine replacement therapy . The cellular telephone intervention received eight counseling sessions delivered via cellular telephone in addition to the usual care components . Smoking-related outcomes were assessed at a 3-month follow-up . The trial had 95 participants and 77 ( 81.0 % ) completed the 3-month follow-up assessment . Analyses indicated biochemically verified point prevalence smoking ab

In [13]:
# 輸出執行時間
excute_time = end - start
print("執行時間：%f 秒" % (excute_time))
print("Inverted index 搜尋的時間:" + str(time_InvertedIndex) + "秒")
print("暴力搜尋時間:" + str(excute_time) + "秒")
print("相差:" + str(excute_time - time_InvertedIndex) + "秒")

執行時間：7.788625 秒
Inverted index 搜尋的時間:0.06647825241088867秒
暴力搜尋時間:7.78862452507019秒
相差:7.722146272659302秒
